In [ ]:
# get file path
import os
os.getcwd()

In [ ]:
# import package (data)

import pandas as pd
import numpy as np
import json

In [ ]:
# import packages (visualization)

import plotly.express as px 
import plotly.graph_objects as go  
import googlemaps

In [ ]:
# There are two ways to plot maps.
# Scatter plot (spot) / Choropleth (area)

# 1) Scatter plot
# We must need longitude and latitude.
# Therefore, we must find longitude and latitude if it doesn't exist.

In [ ]:
# 1-1) Dataset have longitude and latitude

# read data
df = pd.read_csv("listings_NY.csv")
df_map = df[['price', 'latitude', 'longitude']]

df_map['price'] = df_map['price'].str.replace('$', '')
df_map['price'] = df_map['price'].str.replace(',','')
df_map['price'] = df_map['price'].astype(float)

q75_map, q25_map = np.percentile(df_map.price, [75 ,25])
iqr_map = q75_map - q25_map
upper_map = q75_map+1.5*iqr_map
lower_map = q25_map-1.5*iqr_map
df_map = df_map[df_map.price < upper_map]

In [ ]:
# 1-1-1) plotly - px.scatter_mapbox
#        1> do not need mapbox token
#        2> can zoom in/out
#        3> fast loading speed
#        4> cannot change marker size (cuz px.scatter_mapbox doesn't have attribute for it)

# plotly.express.mapbox description
# https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html

# https://plotly.com/python/mapbox-layers/
# https://plotly.com/python/scattermapbox/

# for vscode (if you use jupyter notebook do not run)
import plotly.io as pio
pio.renderers.default = "notebook_connected" 

import plotly.express as px  # px.scatter_mapbox

fig = px.scatter_mapbox(df_map,
                        lat="latitude",
                        lon="longitude",
                        color="price",
                        color_continuous_scale="icefire",
                        zoom=10, # Between 0 and 20. Sets map zoom level.
                        width=900,
                        height=800,
                        #opacity=1,
                        mapbox_style="carto-positron" 
                        #'open-street-map', 'white-bg', 'carto-positron', 'carto-darkmatter', 
                        #'stamen- terrain', 'stamen-toner', 'stamen-watercolor'
                       )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}) # eliminate margin
fig.show()

In [ ]:
# 1-1-2) plotly - go.Scattergeo
#        1> can change marker size
#        2> powerful with big area (usa, world map)
#        3> cannot zoom in/out
#        4> slow loading speed

# https://plotly.com/python/scatter-plots-on-maps/

# plotly.graph_objects.Figure description
# https://plotly.com/python-api-reference/generated/plotly.graph_objects.Figure.html

# plotly.graph_objects.Scattergeo description
# https://plotly.github.io/plotly.py-docs/generated/plotly.graph_objects.Scattergeo.html

import plotly.graph_objects as go  # go.Figure(data=go.Scattergeo())

fig = go.Figure(
      data=go.Scattergeo(lon = df_map['longitude'],
                         lat = df_map['latitude'],
                         mode = 'markers', # markers, text, markers+text
                         text = df_map['price'],
                         marker_color = df_map['price'],
                         marker_size = 1
                        ))

# update layout - if not, plot world map

# fig.update_layout description
# https://plotly.com/python/reference/layout/

# Python Figure Reference: layout.geo description
# https://plotly.com/python/reference/layout/geo/

fig.update_layout(
    margin={"r":0,"t":80,"l":0,"b":0},
    title = "Airbnb in New York",
    width = 1500,
    height = 700,
    autosize = False,
    geo = dict(
        scope = "usa" , # "africa" | "asia" | "europe" | "north america" | "south america" | "usa" | "world"
        fitbounds="locations", # plot visible areas only
        visible=False,   # Sets the default visibility of the base layers
        showland = True,
        resolution = 50, # coast line accuracy (50~110)
    ))
fig.show()

In [ ]:
# 1-1-3) plotly - go.Scattermapbox
#        1> need mapbox token
#        2> can change marker size!
#        3> except 1,2 everthing is same with 1-1-1
#        4> so use this!!!

# plotly.graph_objects.Scattermapbox description
# https://plotly.com/python-api-reference/generated/plotly.graph_objects.Scattermapbox.html
# https://plotly.com/python/scattermapbox/

# https://docs.mapbox.com/help/getting-started/access-tokens/#how-access-tokens-work

import plotly

mapbox_token = 'mapbox token'
px.set_mapbox_access_token(mapbox_token)

fig = go.Figure(go.Scattermapbox(
        lat=df_map['latitude'],
        lon=df_map['longitude'],
        mode='markers',
        text=df_map['price'],
    
        marker=go.scattermapbox.Marker(
            
            color=df_map['price'],
            colorscale="icefire",
            size=2,
            colorbar={"title":"price"}
        )))

fig.update_layout(
    margin={"r":0,"t":80,"l":0,"b":0},
    title = "Airbnb in New York",
    width = 900,
    height = 800,
    mapbox=dict(
        accesstoken=mapbox_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=df_map['latitude'].mean(),
            lon=df_map['longitude'].mean()
        ),
        pitch=0,
        zoom=10
    )
)

fig.show()

In [ ]:
# 1-1-4) Basemap
#        Don't use Basemap because it doesn't support updates anymore.

In [ ]:
# density plot (heatmap)

fig = px.density_mapbox(df_map,
                        lat='latitude',
                        lon='longitude',
                        center=dict(lat=df_map.latitude.mean(), lon=df_map.longitude.mean()), 
                        radius=2, #Sets the radius of influence of each point
                        zoom=10,
                        mapbox_style="open-street-map", 
                        #'open-street-map', 'white-bg', 'carto-positron', 
                        #'carto-darkmatter', 'stamen- terrain', 'stamen-toner', 'stamen-watercolor'
                        width = 900,
                        height = 900
                        )
#fig.update_traces(visible=False)
fig.show()

# https://plotly.com/python/reference/densitymapbox/
# https://plotly.com/python-api-reference/generated/plotly.express.density_mapbox

## heatmap reflect the density, not price exactly.

In [ ]:
# 1-2) Dataset doesn't have longitude and latitude
#      We need googlemap API to get longitude and latitude

# read data and get longitude and latitude

df2 = pd.read_csv("state_unique_temp_111321.csv")

import googlemaps
gmaps_key = googlemaps.Client(key = "googlemap API")

df2["Lat"] = None
df2["Lon"] = None

for i in range(0, len(df2), 1):
    geocode_result = gmaps_key.geocode(df2.iat[i,1]) # location column
    try:
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lon = geocode_result[0]["geometry"]["location"]["lng"]
        df2.iat[i, df2.columns.get_loc("Lat")] = lat
        df2.iat[i, df2.columns.get_loc("Lon")] = lon
    except:
        lat = None
        lon = None

In [ ]:
# After getting coordinates, all processes are same.
# 1-1-2) plotly - go.Scattergeo

import plotly.graph_objects as go 

fig = go.Figure(
      data=go.Scattergeo(lon = df2['Lon'],
                         lat = df2['Lat'],
                         mode = 'markers+text', # markers, text, markers+text
                         text = df2['State'],
                         marker_color = df2['zip_count'],
                         marker_size = 1
                        ))

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    width = 950,
    height = 700,
    autosize = False,
    geo = dict(
        scope = "usa" , # "africa" | "asia" | "europe" | "north america" | "south america" | "usa" | "world"
        showland = True,
        landcolor = "rgb(250, 250, 250)",
        subunitcolor = "rgb(217, 217, 217)",
        countrycolor = "rgb(217, 217, 217)",
        countrywidth = 0.5,
        subunitwidth = 0.5
    ))
fig.show()

In [ ]:
# 2) Choropleth
# We can plot choropleth maps without using longitude and latitude, if packages support area data
# If not, we must need geojson file to plot choropleth maps.

# 2-1) Plot maps without using longitude and latitude

abb = pd.read_csv("state_abb.csv") # get state abbreviation
df2 = df2.drop(df2.columns[0], axis=1)
df3 = pd.merge(abb, df2, on="State", how= "outer")

In [ ]:
# plotly.express.choropleth description
# https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html

fig = px.choropleth(data_frame = df3,
                    locations= "Code", 
                    locationmode="USA-states",
                    scope="usa",                 
                    color="zip_count", 
                    color_continuous_scale="bluered",
#                    color_discrete_map = {"Yes" : "royalblue", 
#                                           "No" : "mediumseagreen", 
#                                           "No fund" : "lightsteelblue" }
                    )

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
#    title_text="count",
    width=950,
    height=700,
#    legend=dict(
#        title="Lockdown",
#        font={'size':25}
#    )
)

fig.add_trace(go.Scattergeo(
        lon = df3['Lon'],
        lat = df3['Lat'],
        mode = 'text',
        showlegend= False,
        text=df3.Code,
        textfont={'color':'white', 'size':8}
        ))
fig.show()

# colormap (single color)
# https://matplotlib.org/stable/gallery/color/named_colors.html

# color scales
# https://plotly.com/python/builtin-colorscales/ 

In [ ]:
# 2-2) Plot maps with geojson file

# https://plotly.com/python/choropleth-maps/

# example 1 (use featureidkey)

df4 = px.data.election()
geojson = px.data.election_geojson()

fig = px.choropleth(df4,
                    geojson=geojson,
                    color="Bergeron",
                    locations="district",
                    featureidkey="properties.district",
                    projection="mercator"
                   )

fig.update_geos(fitbounds="locations", visible=False) # If do not apply this code, plot worldmap
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# example 2 (standard example)
# https://plotly.com/python/choropleth-maps/

df5 = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str}) # why wrote this code? to make fips str type
                                        # (if not, it does not print 0 in front)

from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
fig = px.choropleth(df5,
                    geojson=counties,
                    locations='fips',
                    color='unemp',
                    color_continuous_scale="Viridis",
                    range_color=(0, 12),
                    scope="usa",
                    labels={'unemp':'unemployment rate'}
                   )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# for fast run for plotting (create new window for plotting)

import plotly.io as pio
pio.renderers.default = 'browser'

In [ ]:
# my example

df_csv = pd.read_csv("Strategic_Monitoring_Plan_(SMP).csv")
df_geo = json.load(open("Strategic_Monitoring_Plan_(SMP).geojson"))

In [ ]:
# We must analyze the structure of geojson file first!

df_geo.keys()

## structure

# {'type' : ,
#   'name' : , 
#   'crs' : {'type' :
#             'properties' : {'name': ''}}
#   'features' : [{'type' : 'Feature',
#                  'properties' : {'OBJECTID' 1: ,
#                                  'HUC' : ,
#                                  'WBID' : ,
#                                    ...
#                                  'SHAPELEN' : },
#                  'geometry' : {'type' : ,
#                                'coordinates' : }}
#
#                 {'type' : 'Feature',
#                  'properties' : {'OBJECTID' 2: ,
#                                  'HUC' : ,
#                                  'WBID' : ,
#                                    ...
#                                  'SHAPELEN' : },
#                  'geometry' : {'type' : ,
#                                'coordinates' : }}]}

In [ ]:
# Check there is id kdy in features

df_geo['features'][0].keys() # features have 3 keys (no id key, we need id key for plotting)

In [ ]:
# make id key

df_geo_id = {}
for feature in df_geo['features']:
    feature['id'] = feature['properties']['OBJECTID']
    df_geo_id[feature['properties']['OBJECTID']] = feature['id']

In [ ]:
# The geojson must be unwound before plotting

!pip install geojson-rewind
from geojson_rewind import rewind # for fixing filling error

geo_re = rewind(df_geo,rfc7946=False)

# plotting
fig = px.choropleth(df_csv,
                    geojson=geo_re,
                    locations="OBJECTID", 
                    color = "SHAPELEN",
                    #scope='usa'
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()